# [Opencup](https://www.opencup.gov.it/portale/web/opencup/dettaglio-opendata-complessivo)


## csv2parquet con Dask

In [1]:
import pandas as pd
import os
from dask import dataframe as dd
import numpy as np
os.chdir('C:/users/paolo/Downloads')
name = 'OpenData Complessivo' # 'OpenData Area Sud CSV', 'OpenData Area Centro CSV', 'OpenData Area Nord-Est CSV','OpenData Area Nord-Ovest CSV','OpenData Area Isole CSV'

In [2]:
# D:\duckdb\files\Opencup
req_cols = ['CUP','DESCRIZIONE_SINTETICA_CUP','ANNO_DECISIONE','REGIONE','PROVINCIA',
'PIVA_CODFISCALE_SOG_TITOLARE','SOGGETTO_TITOLARE','DENO_IMPRESA_STABILIMENTO','PIVA_CF_BENEFICIARIO',
            'COSTO_PROGETTO','FINANZIAMENTO_PROGETTO']

dask_df = dd.read_csv('TOTALE.csv',sep='|', usecols=req_cols,
                      dtype={'ANNO_DECISIONE':'int',
                             'PIVA_CODFISCALE_SOG_TITOLARE': 'object'},
                     encoding='utf-8')

In [3]:
dask_df.info()

<class 'dask.dataframe.core.DataFrame'>
Columns: 11 entries, CUP to PIVA_CF_BENEFICIARIO
dtypes: object(8), int32(1), int64(2)

In [4]:
%%time
byState = dask_df.groupby('REGIONE')
byState['COSTO_PROGETTO'].count().compute()

CPU times: total: 39.9 s
Wall time: 1min 1s


REGIONE
ABRUZZO                   500744
BASILICATA                368235
CALABRIA                  772206
CAMPANIA                 1662880
EMILIA-ROMAGNA           1848501
FRIULI-VENEZIA GIULIA     496515
LAZIO                     599552
LIGURIA                   177983
LOMBARDIA                1875940
MARCHE                    527413
MOLISE                     67443
PIEMONTE                 1260502
PUGLIA                    745066
SARDEGNA                 1414074
SICILIA                   681847
TOSCANA                   736223
TRENTINO-ALTO ADIGE       890166
TUTTE                       7453
UMBRIA                    409954
VALLE D'AOSTA             199460
VENETO                   1577342
Name: COSTO_PROGETTO, dtype: int64

### salvare 

In [6]:
dask_df.repartition(npartitions=1).to_parquet('D:/duckdb/files/Opencup', write_index=False) # engine=' pyarrow,fastparquet

## analisi con Polars

In [7]:
import polars as pl

In [9]:
%%time
# leggo il parquet
df= pl.read_parquet('D:/duckdb/files/Opencup/part.0.parquet')

CPU times: total: 6.83 s
Wall time: 3.26 s


In [13]:
df.shape

(16819499, 11)

In [10]:
df.rows

<bound method DataFrame.rows of shape: (16_819_499, 11)
┌──────────────┬──────────────┬──────────────┬─────────────┬───┬─────────────┬─────────────┬─────────────┬─────────────┐
│ CUP          ┆ DESCRIZIONE_ ┆ ANNO_DECISIO ┆ COSTO_PROGE ┆ … ┆ PIVA_CODFIS ┆ SOGGETTO_TI ┆ DENO_IMPRES ┆ PIVA_CF_BEN │
│ ---          ┆ SINTETICA_CU ┆ NE           ┆ TTO         ┆   ┆ CALE_SOG_TI ┆ TOLARE      ┆ A_STABILIME ┆ EFICIARIO   │
│ str          ┆ P            ┆ ---          ┆ ---         ┆   ┆ TOLARE      ┆ ---         ┆ NTO         ┆ ---         │
│              ┆ ---          ┆ i32          ┆ i64         ┆   ┆ ---         ┆ str         ┆ ---         ┆ str         │
│              ┆ str          ┆              ┆             ┆   ┆ str         ┆             ┆ str         ┆             │
╞══════════════╪══════════════╪══════════════╪═════════════╪═══╪═════════════╪═════════════╪═════════════╪═════════════╡
│ B93G99000090 ┆ ITALJOLLY -  ┆ 2001         ┆ 4659990     ┆ … ┆ 80230390587 ┆ MINISTERO   ┆ null

In [11]:
df.filter(pl.col("REGIONE")=='ABRUZZO')

CUP,DESCRIZIONE_SINTETICA_CUP,ANNO_DECISIONE,COSTO_PROGETTO,FINANZIAMENTO_PROGETTO,REGIONE,PROVINCIA,PIVA_CODFISCALE_SOG_TITOLARE,SOGGETTO_TITOLARE,DENO_IMPRESA_STABILIMENTO,PIVA_CF_BENEFICIARIO
str,str,i32,i64,i64,str,str,str,str,str,str
"""C13F1101146000…","""ABITAZIONE**RI…",2011,10000,10000,"""ABRUZZO""","""L'AQUILA""","""80002270660""","""COMUNE DI L'AQ…",null,"""**************…"
"""J51E1300018000…","""SCUOLA MEDIA S…",2013,1150000,1150000,"""ABRUZZO""","""CHIETI""","""00247720691""","""COMUNE DI SAN …",null,"""**************…"
"""F33F1706675000…","""LA MONACO DEBO…",2017,183,183,"""ABRUZZO""","""CHIETI""","""97181460581""","""AGENZIA PER LE…","""LA MONACO DEBO…","""02030740696"""
"""C96I1800015000…","""DI PANGRAZIO M…",2018,1322,1322,"""ABRUZZO""","""L'AQUILA""","""80003170661""","""REGIONE ABRUZZ…","""DI PANGRAZIO M…","""01536050667"""
"""C65B1800144000…","""PASQUALONE DIE…",2018,484,484,"""ABRUZZO""","""PESCARA""","""80003170661""","""REGIONE ABRUZZ…","""PASQUALONE DIE…","""01732740681"""
"""C26I1800326000…","""LOVATO MAURO*P…",2018,5427,5427,"""ABRUZZO""","""CHIETI""","""80003170661""","""REGIONE ABRUZZ…","""LOVATO MAURO""","""01741230690"""
"""F58G1910958000…","""ONOFRILLO ETTO…",2019,85,85,"""ABRUZZO""","""CHIETI""","""97181460581""","""AGENZIA PER LE…","""ONOFRILLO ETTO…","""00795690692"""
"""F38G1912225000…","""MARIOTTI GIUSE…",2019,218,218,"""ABRUZZO""","""CHIETI""","""97181460581""","""AGENZIA PER LE…","""MARIOTTI GIUSE…","""01694920693"""
"""C95C2101024000…","""MD AUTO SRL*CO…",2021,5000,5000,"""ABRUZZO""","""TUTTE""","""80003170661""","""REGIONE ABRUZZ…","""MD AUTO SRL""","""01618050692"""


In [22]:
df.groupby(by='REGIONE').agg(pl.col('FINANZIAMENTO_PROGETTO').sum().alias('totale'))

REGIONE,totale
str,i64
"""CALABRIA""",215380682808
"""LAZIO""",304761333554
"""EMILIA-ROMAGNA…",245223469339
"""TOSCANA""",209811846158
"""TRENTINO-ALTO …",133813621474
"""SARDEGNA""",125330446008
"""PUGLIA""",234374267020
"""PIEMONTE""",335909352463
"""BASILICATA""",101981822045


## analisi con PyArrow

In [1]:
import pyarrow.parquet as pq
import pandas as pd
import glob
import duckdb
import os
os.chdir('D:\\duckdb\\files\\Opencup\\TOT')
 
con = duckdb.connect()
con.execute("PRAGMA threads=2") # enable automatic query parallelization
con.execute("PRAGMA enable_object_cache") # enable caching of parquet metadata

In [2]:
%%time
print(con.execute("SELECT COUNT(*) FROM '*.parquet'").df())

   count_star()
0      15870365
CPU times: total: 0 ns
Wall time: 9.25 ms


In [27]:
print(con.execute("SELECT * FROM '*.parquet' limit 1").df())

               CUP                          DESCRIZIONE_SINTETICA_CUP  \
0  B93G99000090015  ITALJOLLY - COMPAGNIA ITALIANA DEI JOLLY HOTEL...   

   ANNO_DECISIONE  COSTO_PROGETTO  FINANZIAMENTO_PROGETTO  REGIONE PROVINCIA  \
0            2001         4659990                  817290  SICILIA   PALERMO   

  PIVA_CODFISCALE_SOG_TITOLARE                            SOGGETTO_TITOLARE  \
0                  80230390587  MINISTERO DELLE IMPRESE E DEL MADE IN ITALY   

  DENO_IMPRESA_STABILIMENTO PIVA_CF_BENEFICIARIO  
0                       NaN      ***************  
CPU times: total: 15.6 ms
Wall time: 38.4 ms


In [5]:
df = (con.execute("SELECT PIVA_CODFISCALE_SOG_TITOLARE,SOGGETTO_TITOLARE FROM '*.parquet' group by 1,2").df())
df.to_csv('D:/SOGGETTO_TITOLARE.csv', sep='|')

In [14]:
df = (con.execute("SELECT ANNO_DECISIONE,round(count(distinct(CUP))/1000) CUPx1000, round(count(CUP)/1000) opnix1000, \
round(sum(COSTO_PROGETTO)/1000000) COSTO_PROGETTOxM€, round(sum(FINANZIAMENTO_PROGETTO)/1000000)  FINANZIAMENTO_PROGETTOxM€ \
FROM '*.parquet' group by 1 order by 1 desc").df())
df

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

,ANNO_DECISIONE,CUPx1000,opnix1000,COSTO_PROGETTOxM€,FINANZIAMENTO_PROGETTOxM€
0,2023,269.0,585.0,151536.0,104944.0
1,2022,841.0,2232.0,532089.0,459669.0
2,2021,769.0,1920.0,524202.0,472192.0
3,2020,712.0,1836.0,442441.0,410175.0
4,2019,574.0,1827.0,181845.0,150647.0
...,...,...,...,...,...
54,1967,0.0,0.0,3.0,16.0
55,1960,0.0,0.0,0.0,0.0
56,1959,0.0,0.0,0.0,0.0
57,1955,0.0,0.0,0.0,0.0


In [36]:
df = (con.execute("SELECT * FROM '*.parquet' where SOGGETTO_TITOLARE like '%SIMEST%' order by CUP").df())
df

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

,CUP,DESCRIZIONE_SINTETICA_CUP,ANNO_DECISIONE,COSTO_PROGETTO,FINANZIAMENTO_PROGETTO,REGIONE,PROVINCIA,PIVA_CODFISCALE_SOG_TITOLARE,SOGGETTO_TITOLARE,DENO_IMPRESA_STABILIMENTO,PIVA_CF_BENEFICIARIO
0,E11B21007570006,MA GROUP SRL*FINANZIAMENTI PER L'INTERNAZIONALIZZAZIONE DELLE IMPRESE E PER LA TRANSIZIONE DIGITALE*VIA MARCO FURIO BIBACULO 2B,2021,250000,250000,LOMBARDIA,CREMONA,04102891001,SIMEST SPA - ROMA,MA GROUP SRL,01321490193
1,E11B21007580006,FLORIUS SRL*FINANZIAMENTI PER L'INTERNAZIONALIZZAZIONE DELLE IMPRESE E PER LA TRANSIZIONE DIGITALE*VIA REIS ROMOLI 122A,2021,300000,300000,PIEMONTE,TORINO,04102891001,SIMEST SPA - ROMA,FLORIUS SRL,11948870016
2,E11B21007590006,NUOVA BEM SRL*FINANZIAMENTI PER L'INTERNAZIONALIZZAZIONE DELLE IMPRESE E PER LA TRANSIZIONE DIGITALE*VIA ALFONSO LAMARMORA 72,2021,300000,300000,PIEMONTE,TORINO,04102891001,SIMEST SPA - ROMA,NUOVA BEM SRL,04914600285
3,E11B21007600006,ECOVER SRL*FINANZIAMENTI PER L'INTERNAZIONALIZZAZIONE DELLE IMPRESE E PER LA TRANSIZIONE DIGITALE*VIA ADAMELLO 8,2021,100000,100000,LOMBARDIA,BRESCIA,04102891001,SIMEST SPA - ROMA,ECOVER SRL,01926470988
4,E11B21007610006,DAMI SRL*FINANZIAMENTI PER L'INTERNAZIONALIZZAZIONE DELLE IMPRESE E PER LA TRANSIZIONE DIGITALE*STRADA MONTURANESE 1462,2021,300000,300000,MARCHE,FERMO,04102891001,SIMEST SPA - ROMA,DAMI SRL,00412840449
...,...,...,...,...,...,...,...,...,...,...,...
12194,E99J23004450001,TOPFLY SRL*FINANZIAMENTI PER L'INTERNAZIONALIZZAZIONE DELLE IMPRESE*VIA CAPPELLE 15,2023,20000,20000,ABRUZZO,CHIETI,04102891001,SIMEST SPA - ROMA,TOPFLY SRL,02672890692
12195,E99J23004460001,G GROUP SRL*FINANZIAMENTI PER L'INTERNAZIONALIZZAZIONE DELLE IMPRESE*PIAZZA UMBERTO I 23,2023,30000,30000,PUGLIA,TARANTO,04102891001,SIMEST SPA - ROMA,G GROUP SRL,02941790731
12196,E99J23004470001,VAMAG SRL*FINANZIAMENTI PER L'INTERNAZIONALIZZAZIONE DELLE IMPRESE*VIA PASCOLI 15,2023,210000,210000,LOMBARDIA,VARESE,04102891001,SIMEST SPA - ROMA,VAMAG SRL,01234220125
12197,E99J23004480001,FAMILY MARKET SRL*FINANZIAMENTI PER L'INTERNAZIONALIZZAZIONE DELLE IMPRESE*VIA GIOVANNI AMENDOLA 22,2023,378000,378000,LAZIO,ROMA,04102891001,SIMEST SPA - ROMA,FAMILY MARKET SRL,14534251005


In [34]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12199 entries, 0 to 12198
Data columns (total 11 columns):
 #   Column                        Non-Null Count  Dtype 
---  ------                        --------------  ----- 
 0   CUP                           12199 non-null  object
 1   DESCRIZIONE_SINTETICA_CUP     12199 non-null  object
 2   ANNO_DECISIONE                12199 non-null  int32 
 3   COSTO_PROGETTO                12199 non-null  int64 
 4   FINANZIAMENTO_PROGETTO        12199 non-null  int64 
 5   REGIONE                       12199 non-null  object
 6   PROVINCIA                     12199 non-null  object
 7   PIVA_CODFISCALE_SOG_TITOLARE  12199 non-null  object
 8   SOGGETTO_TITOLARE             12199 non-null  object
 9   DENO_IMPRESA_STABILIMENTO     12199 non-null  object
 10  PIVA_CF_BENEFICIARIO          12199 non-null  object
dtypes: int32(1), int64(2), object(8)
memory usage: 1000.8+ KB


In [35]:
df.to_csv('D:/Opencup_Simest.csv',sep='|', index= False)

# creare un DuckDB DB

In [ ]:
import duckdb
import os
os.chdir('D:')
conn = duckdb.connect(database='Opendata.duckdb', read_only=False)
conn.execute(f'''CREATE TABLE Opencup AS SELECT * FROM read_parquet('D:/duckdb/files/Opencup/TOT/*.parquet');''').df()

## backup

In [13]:
dask_df = dd.read_csv('TOTALE.csv',sep='|') # ,dtype=object)

In [ ]:
#read data in chunks of 1 million rows at a time
chunk = pd.read_csv('TOTALE.csv',sep='|',chunksize=1000000)

pd_df = pd.concat(chunk)

In [ ]:
df = pd.read_csv(name+'.zip', compression='zip', sep='|', quotechar='"', 
                 dtype = {'PIVA_CODFISCALE_SOG_TITOLARE' : object,
                          'COD_GRUPPO_ATECO': object,
                          'NUM_CIVICO_SOG_TITOLARE': object,
                          'CAP_SOGGETTO_TITOLARE' : object})

In [18]:
df.shape

(Delayed('int-1a08948b-8b19-4036-a432-3ed13cd53ebc'), 91)

In [12]:
pd.set_option('display.max_column', None)
df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20 entries, 0 to 19
Data columns (total 91 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   CUP                             20 non-null     object 
 1   DESCRIZIONE_SINTETICA_CUP       20 non-null     object 
 2   ANNO_DECISIONE                  20 non-null     int64  
 3   STATO_PROGETTO                  20 non-null     object 
 4   COSTO_PROGETTO                  20 non-null     int64  
 5   FINANZIAMENTO_PROGETTO          20 non-null     int64  
 6   CODICE_STATO                    20 non-null     int64  
 7   STATO                           20 non-null     object 
 8   CODICE_AREA_GEOGRAFICA          20 non-null     object 
 9   AREA_GEOGRAFICA                 20 non-null     object 
 10  CODICE_REGIONE                  20 non-null     int64  
 11  REGIONE                         20 non-null     object 
 12  CODICE_PROVINCIA                20 non

### usare i parquet - prima alternativa

In [ ]:
# con pandas è lento
%%time
con = duckdb.connect()
os.chdir('D:/duckdb/files/Opencup/20230301')
opencup = con.execute(f'''select * from read_parquet('*.parquet') ''').df()

### usare i parquet - seconda alternativa con Pyarrow

In [ ]:
import pyarrow.parquet as pq
import pandas as pd
import os
import duckdb
os.chdir('D:/duckdb/files/Opencup/20230101')

# some DuckDB setup 
con = duckdb.connect()
# enable automatic query parallelization
con.execute("PRAGMA threads=2")
# enable caching of parquet metadata
con.execute("PRAGMA enable_object_cache")

opencup = con.execute(f'''select distinct(PIVA_CF_BENEFICIARIO) CF, DENO_IMPRESA_STABILIMENTO impresa
from read_parquet('*.parquet') GROUP BY 1,2 order by 2''').df()

opencup.insert(2,"fonte","opencup") # insert column
opencup['aggiornamento'] = pd.Timestamp('now').date() # insert column with today date

# opencup = opencup.sort_values(by=['impresa'], inplace=True) # sort
opencup = opencup[opencup.CF != '***************']
# opencup.head()
opencup.to_parquet('D:/duckdb/files/CF/opencup.parquet')

#### OBIETTIVO A : creo un dataframe contenente codici fiscali univoci e relativo nome dell'impresa

In [ ]:
opencupSUB = opencup.filter(['PIVA_CF_BENEFICIARIO','DENO_IMPRESA_STABILIMENTO'], axis=1) # nuovo dataframe con alcune colonne
opencupSUB = opencupSUB.dropna()

dict = {'PIVA_CF_BENEFICIARIO': 'CF', 'DENO_IMPRESA_STABILIMENTO': 'impresa'} # create a dictionary with old and new name
opencupSUB.rename(columns=dict,inplace=True) # change column header
opencupSUB['fonte'] = 'Opencup' # add column with standard value
# opencupSUB['aggiornamento'] = '2023-02-24' 
# opencupSUB['aggiornamento'] = pd.to_datetime(opencupSUB['aggiornamento']) # object to date
opencupSUB['aggiornamento'] = pd.Timestamp('now').date()
opencupSUB = opencupSUB[opencupSUB['CF'] != '***************'] # rimuovo le righe che hanno un determinato valore

# opencupSUB.to_csv('D:\\Opencup.csv', sep='\t', index=False)
opencupSUB.to_parquet('D:/duckdb/files/CF/Opencup.parquet')


#### OBIETTIVO B : creo un dataframe con i CUP e i relativi beneficiari

In [ ]:
opencupSUB = opencup.filter(['CUP', 'PIVA_CF_BENEFICIARIO','DENO_IMPRESA_STABILIMENTO'], axis=1) # nuovo dataframe con alcune colonne
opencupSUB = opencupSUB.dropna()
opencupSUB = opencupSUB[opencupSUB['PIVA_CF_BENEFICIARIO'] != '***************']
dict = {'PIVA_CF_BENEFICIARIO': 'CF', 'DENO_IMPRESA_STABILIMENTO': 'impresa'} # create a dictionary with old and new name
opencupSUB.rename(columns=dict,inplace=True) # change column header

opencupSUB.to_csv('D:/duckdb/files/Opencup/CUP.csv', sep='\t', index=False)

In [ ]:
## prova 2